In [1]:
!pip install transformers
!pip install datasets
!pip install rouge_score
!pip install nltk torch

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
pip install --upgrade numpy scipy pandas


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 13.1 MB 4.8 MB/s eta 0:00:01�████████████       | 10.3 MB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 346 kB 99.2 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install pandas==1.3.3  # Known to work well with most packages
# or try the latest stable version if necessary
!pip install pandas --upgrade
!pip install numpy==1.21.2 datasets --upgrade

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached pandas-2.2.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.1 MB)
  Using cached numpy-2.0.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.5 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.2
    Uninstalling numpy-1.21.2:
      Successfully uninstalled numpy-1.21.2
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.3
    Uninstalling pandas-1.3.3:
      Successfully uninstalled pandas-1.3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.3.0 requires daal==2021.2.3, which is not instal

In [3]:
# Imports
from datasets import load_dataset
from transformers import PegasusTokenizer, PegasusForConditionalGeneration, AdamW
from rouge_score import rouge_scorer
from torch.utils.data import DataLoader
from torch import nn
import torch
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from torch.cuda.amp import autocast, GradScaler
import os
import gc  # Import garbage collector for memory management

# Ensure necessary NLTK data is downloaded
nltk.download('stopwords')
nltk.download('wordnet')

# Device setting
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


/home/desai226/.local/lib/python3.9/site-packages/transformers/loss/loss_for_object_detection.py:28: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 1.21.2)
  from scipy.optimize import linear_sum_assignment


Using device: cuda


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/desai226/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/desai226/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
# Load and Prepare PEGASUS Model
pegasus_model_name = 'google/pegasus-large'
print(f"Loading PEGASUS model: {pegasus_model_name}")
tokenizer = PegasusTokenizer.from_pretrained(pegasus_model_name)
model = PegasusForConditionalGeneration.from_pretrained(pegasus_model_name).to(device)


Loading PEGASUS model: google/pegasus-large


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Preprocessing function
def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    words = text.split()
    words = [lemmatizer.lemmatize(word.lower()) for word in words if word.lower() not in stop_words]
    return ' '.join(words)

# Function to remove rows with missing data
def remove_missing_data(dataset, columns):
    return dataset.filter(lambda x: all(x[col] is not None for col in columns))

# Function to load and clean dataset
def load_and_clean_data():
    train_data = load_dataset("ragha92/FNS_Summarization", split="train")
    validation_data = load_dataset("ragha92/FNS_Summarization", split="validation")
    test_data = load_dataset("ragha92/FNS_Summarization", split="test")

    columns = ['Annual Reports', 'Gold Summaries']
    train_data = remove_missing_data(train_data, columns)
    validation_data = remove_missing_data(validation_data, columns)
    test_data = remove_missing_data(test_data, columns)

    return train_data, validation_data, test_data


In [10]:
# Summarization function for PEGASUS with updated hyperparameters
def summarize_text_pegasus(article):
    input_ids = tokenizer.encode(article, return_tensors="pt", max_length=512, truncation=True).to(device)
    summary_ids = model.generate(
        input_ids,
        max_length=512,
        min_length=50,
        length_penalty=1.5,
        num_beams=3,
        early_stopping=True
    )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Checkpoint save/load functions
def save_checkpoint(model, optimizer, epoch, path="pegasus_checkpoint.pt"):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, path)
    print(f"Checkpoint saved at epoch {epoch}")

def load_checkpoint(optimizer, path="pegasus_checkpoint.pt"):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    print(f"Checkpoint loaded. Starting from epoch {epoch + 1}")
    return epoch


In [11]:
# Update fine_tune_model to pass the optimizer to load_checkpoint
def fine_tune_model(train_data, epochs=3, learning_rate=5e-5, save_interval=1):
    model.train()
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    dataloader = DataLoader(train_data, batch_size=2, shuffle=True)  # Adjust batch size as needed

    starting_epoch = 0
    if "pegasus_checkpoint.pt" in os.listdir():  # Check if a checkpoint exists
        starting_epoch = load_checkpoint(optimizer, path="pegasus_checkpoint.pt")

    for epoch in range(starting_epoch, epochs):
        total_loss = 0
        for batch_idx, batch in enumerate(dataloader):
            optimizer.zero_grad()
            input_texts = batch['Annual Reports']
            target_summaries = batch['Gold Summaries']
            inputs = tokenizer(input_texts, return_tensors="pt", max_length=512, truncation=True, padding=True).input_ids.to(device)
            targets = tokenizer(target_summaries, return_tensors="pt", max_length=512, truncation=True, padding=True).input_ids.to(device)

            outputs = model(input_ids=inputs, labels=targets)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            # Clear out unnecessary memory
            del inputs, targets, outputs, loss
            gc.collect()

        if (epoch + 1) % save_interval == 0:
            save_checkpoint(model, optimizer, epoch, path="pegasus_checkpoint.pt")

        print(f"Epoch {epoch+1}/{epochs} Average Loss: {total_loss / len(dataloader)}")


In [12]:
# Evaluation using ROUGE and BLEU with smoothing
def evaluate_model(dataset):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = {'rouge1': [], 'rouge2': [], 'rougeL': [], 'bleu': []}
    smooth = SmoothingFunction().method4

    for item in dataset:
        article = preprocess_text(item['Annual Reports'])
        gold_summary = preprocess_text(item['Gold Summaries'])
        if article:
            generated_summary = summarize_text_pegasus(article)
            score = scorer.score(gold_summary, generated_summary)
            scores['rouge1'].append(score['rouge1'].fmeasure)
            scores['rouge2'].append(score['rouge2'].fmeasure)
            scores['rougeL'].append(score['rougeL'].fmeasure)
            reference = gold_summary.split()
            candidate = generated_summary.split()
            bleu_score = sentence_bleu([reference], candidate, smoothing_function=smooth)
            scores['bleu'].append(bleu_score)

    avg_rouge1 = sum(scores['rouge1']) / len(scores['rouge1'])
    avg_rouge2 = sum(scores['rouge2']) / len(scores['rouge2'])
    avg_rougeL = sum(scores['rougeL']) / len(scores['rougeL'])
    avg_bleu = sum(scores['bleu']) / len(scores['bleu'])
    return avg_rouge1, avg_rouge2, avg_rougeL, avg_bleu


In [13]:
# Loading and cleaning datasets
print("Loading and cleaning datasets...")
train_data, validation_data, test_data = load_and_clean_data()

# Fine-tuning model
print("Fine-tuning model...")
fine_tune_model(train_data, epochs=3)

# Evaluating on the validation set
print("Evaluating on the validation set...")
avg_rouge1, avg_rouge2, avg_rougeL, avg_bleu = evaluate_model(validation_data)
print(f"Validation ROUGE-1: {avg_rouge1}, ROUGE-2: {avg_rouge2}, ROUGE-L: {avg_rougeL}, BLEU: {avg_bleu}")

# Evaluating on the test dataset
print("Evaluating on the test dataset...")
avg_rouge1, avg_rouge2, avg_rougeL, avg_bleu = evaluate_model(test_data)
print(f"Test Set ROUGE-1: {avg_rouge1}")
print(f"Test Set ROUGE-2: {avg_rouge2}")
print(f"Test Set ROUGE-L: {avg_rougeL}")
print(f"Test Set BLEU: {avg_bleu}")


Loading and cleaning datasets...
Fine-tuning model...


/home/desai226/.local/lib/python3.9/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/tmp/ipykernel_92911/3931150796.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serializat

Checkpoint loaded. Starting from epoch 3


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (0) is identical to the `bos_token_id` (0), `eos_token_id` (1), or the `sep_token_id` (None), and your input is not padded.


Checkpoint saved at epoch 2
Epoch 3/3 Average Loss: 2.4914135535202413
Evaluating on the validation set...
Validation ROUGE-1: 0.17869484656639414, ROUGE-2: 0.09531901939864897, ROUGE-L: 0.13108153377083526, BLEU: 0.04686865150276145
Evaluating on the test dataset...
Test Set ROUGE-1: 0.1835236997195584
Test Set ROUGE-2: 0.10053553473877172
Test Set ROUGE-L: 0.1327699621873767
Test Set BLEU: 0.051151248683789376


In [6]:
!pip install sentencepiece

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.3 MB 2.2 MB/s eta 0:00:01


In [3]:
pip list

Package                            Version
---------------------------------- --------------------
absl-py                            2.1.0
aiohappyeyeballs                   2.4.3
aiohttp                            3.10.10
aiosignal                          1.3.1
alabaster                          0.7.12
alembic                            1.7.5
anaconda-client                    1.9.0
anaconda-navigator                 2.1.1
anaconda-project                   0.10.1
annotated-types                    0.7.0
anyascii                           0.3.2
anyio                              3.5.0
appdirs                            1.4.4
argh                               0.26.2
argon2-cffi                        20.1.0
arrow                              0.13.1
asn1crypto                         1.4.0
astroid                            2.6.6
astropy                            4.3.1
asttokens                          2.4.1
async-generator                    1.10
async-timeout                     